# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120500e+02     1.519493e+00
 * time: 0.8142170906066895
     1     1.093979e+01     9.124571e-01
 * time: 2.6485321521759033
     2    -1.130265e+01     9.850673e-01
 * time: 3.348690986633301
     3    -3.379510e+01     7.717721e-01
 * time: 4.450177192687988
     4    -4.728668e+01     5.852969e-01
 * time: 5.54110312461853
     5    -5.687983e+01     2.009011e-01
 * time: 6.532520055770874
     6    -5.977864e+01     1.583991e-01
 * time: 7.233177185058594
     7    -6.085745e+01     5.029478e-02
 * time: 7.902115106582642
     8    -6.126237e+01     8.094122e-02
 * time: 8.570109128952026
     9    -6.156263e+01     4.185138e-02
 * time: 9.261337041854858
    10    -6.178793e+01     3.454875e-02
 * time: 10.022821187973022
    11    -6.195486e+01     2.787196e-02
 * time: 10.740102052688599
    12    -6.202156e+01     2.412186e-02
 * time: 11.437160015106201
    13    -6.209255e+01     1.775297e-02
 * time: 12.094397068023682
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671079
    AtomicLocal         -18.8557691
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336825

    total               -62.261666453999
